# Using with a spreadsheet-based configuration

`ecodynelec` offers the possibility to configure the execution via a spreadsheet, for more user-friendly handling. This tutorial shows the different steps and possibilities in this regard. As `ecodynelec` is firstly designed for research purposes, there also is the possibility to entirely rely on a python-based configuration , as developed  in the "[Using fully with Python](https://ecodynelec.readthedocs.io/en/latest/examples/with_python.html)" tutorial.

## Initialization

To download and install `ecodynelec` to being used as a python package, the user is referred to either the [getting started tutorial](https://ecodynelec.readthedocs.io/en/latest/examples/getting_started.html).

## Configuration

The configuration of `ecodynelec` is to be handled in a spreadsheet. Figure 1-3 show the three sections of the spreadsheet used in this example. The spreadsheet can be downloaded from the [github repository](https://github.com/LESBAT-HEIG-VD/EcoDynElec/raw/main/examples/Spreadsheet_example.xlsx).

The configuration is composed of 3 parts. The detail about the meaning of each is developed in the [input data section](https://ecodynelec.readthedocs.io/en/latest/data_input/parameters.html).

![Main Tab Spreadsheet](https://github.com/LESBAT-HEIG-VD/EcoDynElec/blob/main/docs/examples/images/ParameterExcel_ParamExample.png?raw=true)\
*Figure 1: Spreadsheet for configuration: main Parameter tab*

the <u>first tab</u> (*Figure 1*) contains the elements to configure the execution itself.

![Filepath Tab Spreadsheet](https://github.com/LESBAT-HEIG-VD/EcoDynElec/blob/main/docs/examples/images/ParameterExcel_PathsExample.png?raw=true)\
*Figure 2: Spreadsheet for configuration: Paths tab*

the <u>second tab</u> (*Figure 2*) deals with all paths to information files, directory containing information, or where to write and save information before, during and after the computation. **Note** that, for the `generation`, `exchanges` and `savedir` directories fields, the specified directory *will be created if it does not already exist*. For every other file path element, *a default file* is used if the field is left blank, and an error is returned if the information passed does not correspond to any existing file on your local machine.

![Server tab Spreadsheet](https://github.com/LESBAT-HEIG-VD/EcoDynElec/blob/main/docs/examples/images/ParameterExcel_ServerExample.png?raw=true)\
*Figure 3: Spreadsheet for configuration: Server tab*

the <u>third tab</u> (*Figure 3*) deals with information related to the ENTSO-E server, as electricity data from the ENTSO-E server is at the center of `ecodynelec`. More on this topic is covered on the next paragraph and on the dedicated [downloading tutorial](https://ecodynelec.readthedocs.io/en/latest/examples/downloading.html). **Note** that per default, the field `use server` is set to FALSE, i.e. no download will occur. Also note that the `username` field is supposed to be an email address.

### Downloading Entso-E data

The [downloading tutorial](https://ecodynelec.readthedocs.io/en/latest/examples/downloading.html) covers the specificities about how to download the ENTSO-E data or include the download as part of the `ecodynelec` pipeline execution. This feature is not triggered per default and `ecodynelec` is expecting to find already downloaded ENTSO-E files.

## Execution

`ecodynelec` is build out of a myriad of modules that can be used relatively independently, under the condition that inputs data is shaped the correct way. Fortunately, the entire pipeline starting from a set of parameters and computing down to the calculation of impact metrics.

The usage of this entire pipeline is demonstrated below. This pipeline allows to save results into files (c.f. paragraph on [configuration](https://ecodynelec.readthedocs.io/en/latest/examples/with_spreadsheet.html#configuration)). However results are also always returned for further in-script use. These results are stored in the `impacts` variable for later paragraphs in this tutorial.

In [ ]:
from ecodynelec.pipelines import execute

In [ ]:
from ecodynelec.parameter import Parameter
p = Parameter(excel='./Spreadsheet_download.xlsx')
print('p is ', p)

In [ ]:
from ecodynelec.progress_info import ProgressInfo

impacts = execute(config="./Spreadsheet_download.xlsx", progress_bar=ProgressInfo())

## Outcome and Visualization

The outcome is stored in files and returned for further in-script use. In the previous section, results were stored in the `impacts` variable. The current section highlights the content returned and shows some basic possibilities for data visualization.

In [ ]:
import numpy as np
import pandas as pd

### Description of the outcome

The `impacts` variable contains a collection of tables. This collection is a `dict` with one `Global` key, and one other key per impact category:

In [ ]:
print(impacts.keys())

The `Global` table is the *sum across all technologies* for each index, as it is shown for the first few time steps:

In [ ]:
display(impacts['Global'].head())

The other tables are, for each impact category, the breakdown into all possible sources:

In [ ]:
for i in impacts: # Iterate for all impact categories
    if i=='Global': continue; # Skip the Global, already visualized above.
    
    print(f"#############\nimpacts for {i}:")
    display( impacts[i].head(3).T ) # Transpose table for readability

### Group per country

The following piece of code suggests a basic visualization of the Climate Change category, grouping the results per country of origin of the tracked electricity.

In [ ]:
def compute_per_country(results):
    """Function to group results per country"""
    countries = np.unique([c.split("_")[-1] for c in results.columns]) # List of countries
    
    per_country = []
    for c in countries:
        cols = [k for k in results.columns if k[-3:]==f"_{c}"]
        per_country.append(pd.Series(results.loc[:,cols].sum(axis=1), name=c))
        
    return pd.concat(per_country,axis=1)

In [ ]:
gwp_per_country = compute_per_country(impacts['Climate Change']) # Group Climate Change index impacts per country
gwp_per_country.plot.area(figsize=(12,4), legend='reverse', color=['r','w','y','b','c','k'],
                          title="Some visualization of the Climate Change index aggregated per country"); # Build the graph

### Group per production type

The following piece of code suggests a basic visualization of the Climate Change category, grouping the results per technology of origin of the tracked electricity.

In [ ]:
def compute_per_type(results):
    """Function to group datasets per type of unit, regardless of the country of origin"""
    unit_list = np.unique([k[:-3] if k[-3]=="_" else k for k in results.columns]) # List the different production units
    
    per_unit = []
    for u in unit_list:
        cols = [k for k in results.columns if k[:-3]==u] # collect the useful columns
        per_unit.append(pd.Series(results.loc[:,cols].sum(axis=1), name=u)) # aggregate

    return pd.concat(per_unit,axis=1)

In [ ]:
es13_per_type = compute_per_type(impacts['Climate Change']) # Group Climate Change index impacts per country
es13_per_type.plot.area(figsize=(12,8), legend='reverse',
                        title="Some visualization of the Climate Change index aggregated per source"); # Build the graph